In [ ]:
import random
from math import floor
fp_a = open('DIVIDE_DAT_A_S0I8F0.dat', 'w')
fp_b = open('DIVIDE_DAT_B_S0I8F0.dat', 'w')
fp_c = open('DIVIDE_DAT_C_S0I8F8.dat', 'w')
j_max = 0
k_max = 0
t_max = 0

DATA_WD = 8
for a in range(2**DATA_WD):
    for b in range(2**DATA_WD):
        if b != 0:
            # full precision
            shift = DATA_WD
            scale = 2 ** DATA_WD
            c_target = a / b
            c_target = floor(c_target * scale + 0.5)

            # directly use "/"
            c_calc = 0
            shift = DATA_WD
            scale = 2 ** DATA_WD
            c_calc = a * scale / b
            c_calc = floor(c_calc + 0.5)
            if (c_calc != c_target):
                print("ERROR @ directly use "/"")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder restoring method
            c_calc = 0
            shiftT = DATA_WD + 1
            scaleT = 2 ** shiftT
            shiftB = DATA_WD * 2
            scaleB = 2 ** shiftB
            t = a * scaleT
            c_calc = 0
            for idxBit in range(shiftB + 1):
                t = t - b * scaleB
                if t < 0:
                    t = t + b * scaleB
                    c_calc = c_calc * 2
                else:
                    c_calc = c_calc * 2 + 1
                t = t * 2
            c_calc = floor(c_calc / 2 + 0.5)
            if (c_calc != c_target):
                print("ERROR @ remainder restoring method")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use remainder non-restoring method
            c_calc = 0
            shiftT = DATA_WD + 1
            scaleT = 2 ** shiftT
            shiftB = DATA_WD * 2
            scaleB = 2 ** shiftB
            t = a * scaleT
            t = t - b * scaleB
            c_calc = 0
            for idxBit in range(shiftB + 1):
                t = t * 2
                if t < 0:
                    t = t + b * scaleB
                    c_calc = c_calc * 2
                else:
                    t = t - b * scaleB
                    c_calc = c_calc * 2 + 1
            c_calc = floor(c_calc / 2 + 0.5)
            if (c_calc != c_target):
                print("ERROR @ remainder non-restoring method")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use LUT
            c_calc = 0
            shift = DATA_WD + 16
            scale = 2 ** shift
            r = floor(1 / b * scale + 0.5)
            c_calc = floor(a * r / scale * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ LUT")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # binary search
            c_calc = 0
            numBit = DATA_WD + 17
            scale = 2 ** numBit
            r = 0
            for idxBit in range(numBit):
                r = r + (2 ** (-1 - idxBit))
                if r * b > 1:
                    r = r - (2 ** (-1 - idxBit))
            r = floor(r * scale + 0.5)
            c_calc = floor(a * r / scale * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ binary search")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use Newton-Raphson iteraion (with full precision)
            c_calc = 0
            NUMB_ITER = 12
            t = 1 / (2 ** DATA_WD)
            for idxItr in range(NUMB_ITER):
                t = t * (2 - b * t)
            c_calc = floor(a * t * (2 ** DATA_WD) + 0.5)
            if (c_calc != c_target):
                print("ERROR @ Newton-Raphson iteraion (with full precision)")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break

            # use Newton-Raphson iteraion
            c_calc = 0
            NUMB_ITER = 12
            shift = DATA_WD + 16
            scale = 2 ** shift
            t = 1 / (2 ** DATA_WD)
            for idxItr in range(NUMB_ITER):
                #print(f"{idxItr}")
                j = b * t
                j = floor(j * scale + 0.5) / scale
                if j_max < j:
                    j_max = j
                #print(f"j: {j * 2 ** 24}")
                k = 2 - j
                if k_max < k:
                    k_max = k
                #print(f"k: {k * 2 ** 24}")
                t = t * k
                t = floor(t * scale + 0.5) / scale
                if t_max < t:
                    t_max = t
                #print(f"t: {t * 2 ** 24}")
            c_calc = floor(a * t * (2 ** DATA_WD) + 0.5)
            fp_a.write("{:x}\n".format(a))
            fp_b.write("{:x}\n".format(b))
            fp_c.write("{:x}\n".format(c_calc))
            if (c_calc != c_target):
                print("ERROR @ Newton-Raphson iteraion")
                print(f"{a} / {b} = {c_target}")
                print(f"{a} / {b} = {c_calc}")
                break
    if (c_calc != c_target):
        break
print(f"max of j, k, t is: {j_max}, {k_max}, {t_max}")
print("DONE!")
fp_a.close()
fp_b.close()
fp_c.close()